In [3]:
import pandas as pd 
import numpy as np
from pandas import Series

In [4]:
#Open filtered fie from control data from diseasea data
df = pd.read_csv('Asthma_final_variant.txt', sep = "\t")

In [5]:
#To samples contains this mutation
df['Sample_Count'] = (df.iloc[:,9:] != './.:.:.:.:.').sum(axis=1)

In [6]:
df['Sample_Count'].head()

0    2
1    1
2    1
3    4
4    1
Name: Sample_Count, dtype: int64

In [7]:
# Remove columns starting with SRR from main variable
df = df[df.columns.drop(list(df.filter(regex='SRR')))]

In [8]:
df.head()

,CHROM,POS,ID,REF,ALT,QUAL,FILTER,INFO,FORMAT,Sample_Count
0,chr1,14522,.,G,A,118.64,PASS,ExcessHet=3.0103;FS=0;MQ=60;QD=33.16;SOR=2.303...,GT:AD:DP:GQ:PL,2
1,chr1,14590,.,G,A,45.64,PASS,BaseQRankSum=-1.15;ExcessHet=3.0103;FS=0;MQ=60...,GT:AD:DP:GQ:PL,1
2,chr1,14673,.,G,C,71.32,PASS,ExcessHet=3.0103;FS=0;MQ=60;QD=25.36;SOR=2.303...,GT:AD:DP:GQ:PL,1
3,chr1,15015,.,G,C,259.65,PASS,ExcessHet=3.0103;FS=0;MQ=60;QD=34.61;SOR=2.833...,GT:AD:DP:GQ:PL,4
4,chr1,17385,.,G,A,57.64,PASS,BaseQRankSum=-0.674;ExcessHet=3.0103;FS=0;MQ=6...,GT:AD:DP:GQ:PL,1


In [9]:
len(df.axes[0]) 

1125712

In [10]:
# info column have many consequences seperated by comma for one mutation to split (comma) that add to next next row by keeping all data
s = df['INFO'].str.split(',').apply(Series, 1).stack() # split by comma
s.index = s.index.droplevel(-1) # index splitted column
s.name = 'INFO' # rename column as INFO

In [11]:
len(s.axes[0]) 

6869861

In [12]:
s.head()

0    ExcessHet=3.0103;FS=0;MQ=60;QD=33.16;SOR=2.303...
0    A|downstream_gene_variant|MODIFIER|DDX11L1|ENS...
0    A|intron_variant&non_coding_transcript_variant...
0    A|downstream_gene_variant|MODIFIER|MIR6859-1|E...
0    A|regulatory_region_variant|MODIFIER|||Regulat...
Name: INFO, dtype: object

In [13]:
#To delete orignal INFO column from main df file
del df['INFO']

In [14]:
# join spliited info column with main df dataframe
df1 = df.join(s)

In [15]:
df1.head()

,CHROM,POS,ID,REF,ALT,QUAL,FILTER,FORMAT,Sample_Count,INFO
0,chr1,14522,.,G,A,118.64,PASS,GT:AD:DP:GQ:PL,2,ExcessHet=3.0103;FS=0;MQ=60;QD=33.16;SOR=2.303...
0,chr1,14522,.,G,A,118.64,PASS,GT:AD:DP:GQ:PL,2,A|downstream_gene_variant|MODIFIER|DDX11L1|ENS...
0,chr1,14522,.,G,A,118.64,PASS,GT:AD:DP:GQ:PL,2,A|intron_variant&non_coding_transcript_variant...
0,chr1,14522,.,G,A,118.64,PASS,GT:AD:DP:GQ:PL,2,A|downstream_gene_variant|MODIFIER|MIR6859-1|E...
0,chr1,14522,.,G,A,118.64,PASS,GT:AD:DP:GQ:PL,2,A|regulatory_region_variant|MODIFIER|||Regulat...


In [16]:
# Again split INFo column by bar(|) to seperate all the colums
k = df1.INFO.str.split("|",expand=True,)

In [17]:
k.head()

,0,1,2,3,4,5,6,7,8,9,...,54,55,56,57,58,59,60,61,62,63
0,ExcessHet=3.0103;FS=0;MQ=60;QD=33.16;SOR=2.303...,downstream_gene_variant,MODIFIER,DDX11L1,ENSG00000223972,Transcript,ENST00000450305,transcribed_unprocessed_pseudogene,,,...,,,,,,,,,,
0,A,downstream_gene_variant,MODIFIER,DDX11L1,ENSG00000223972,Transcript,ENST00000456328,processed_transcript,,,...,,,,,,,,,,
0,A,intron_variant&non_coding_transcript_variant,MODIFIER,WASH7P,ENSG00000227232,Transcript,ENST00000488147,unprocessed_pseudogene,,10/10,...,,,,,,,,,,
0,A,downstream_gene_variant,MODIFIER,MIR6859-1,ENSG00000278267,Transcript,ENST00000619216,miRNA,,,...,,,,,,,,,,
0,A,regulatory_region_variant,MODIFIER,,,RegulatoryFeature,ENSR00000000002,open_chromatin_region,,,...,,,,,,,,,,;BaseQRankSum=0;MQRankSum=0;ReadPosRankSum=0;D...


In [18]:
#To add column names to splitted INFO columns
k.columns = ["Allele","Consequence","IMPACT","SYMBOL","Gene","Feature_type","Feature","BIOTYPE","EXON","INTRON","HGVSc","HGVSp","cDNA_position","CDS_position","Protein_position","Amino_acids","Codons","Existing_variation","DISTANCE","STRAND","FLAGS","VARIANT_CLASS","SYMBOL_SOURCE","HGNC_ID","CANONICAL","TSL","APPRIS","CCDS","ENSP","SWISSPROT","TREMBL","UNIPARC","GENE_PHENO","SIFT","PolyPhen","DOMAINS","HGVS_OFFSET","AF","AFR_AF","AMR_AF","EAS_AF","EUR_AF","SAS_AF","AA_AF","EA_AF","ExAC_AF","ExAC_Adj_AF","ExAC_AFR_AF","ExAC_AMR_AF","ExAC_EAS_AF","ExAC_FIN_AF","ExAC_NFE_AF","ExAC_OTH_AF","ExAC_SAS_AF","MAX_AF","MAX_AF_POPS","CLIN_SIG","SOMATIC","PHENO","PUBMED","MOTIF_NAME","MOTIF_POS","HIGH_INF_POS","MOTIF_SCORE_CHANGE"]

In [19]:
k.head()

,Allele,Consequence,IMPACT,SYMBOL,Gene,Feature_type,Feature,BIOTYPE,EXON,INTRON,...,MAX_AF,MAX_AF_POPS,CLIN_SIG,SOMATIC,PHENO,PUBMED,MOTIF_NAME,MOTIF_POS,HIGH_INF_POS,MOTIF_SCORE_CHANGE
0,ExcessHet=3.0103;FS=0;MQ=60;QD=33.16;SOR=2.303...,downstream_gene_variant,MODIFIER,DDX11L1,ENSG00000223972,Transcript,ENST00000450305,transcribed_unprocessed_pseudogene,,,...,,,,,,,,,,
0,A,downstream_gene_variant,MODIFIER,DDX11L1,ENSG00000223972,Transcript,ENST00000456328,processed_transcript,,,...,,,,,,,,,,
0,A,intron_variant&non_coding_transcript_variant,MODIFIER,WASH7P,ENSG00000227232,Transcript,ENST00000488147,unprocessed_pseudogene,,10/10,...,,,,,,,,,,
0,A,downstream_gene_variant,MODIFIER,MIR6859-1,ENSG00000278267,Transcript,ENST00000619216,miRNA,,,...,,,,,,,,,,
0,A,regulatory_region_variant,MODIFIER,,,RegulatoryFeature,ENSR00000000002,open_chromatin_region,,,...,,,,,,,,,,;BaseQRankSum=0;MQRankSum=0;ReadPosRankSum=0;D...


In [20]:
# To filter only important columns from splitted INFO column and saved in p variable
p = k[["Consequence","SYMBOL","Gene","Feature_type","Feature","BIOTYPE","EXON","INTRON","Existing_variation","SIFT","PolyPhen","AF","ExAC_AF"]]

In [21]:
p.head()

,Consequence,SYMBOL,Gene,Feature_type,Feature,BIOTYPE,EXON,INTRON,Existing_variation,SIFT,PolyPhen,AF,ExAC_AF
0,downstream_gene_variant,DDX11L1,ENSG00000223972,Transcript,ENST00000450305,transcribed_unprocessed_pseudogene,,,,,,,
0,downstream_gene_variant,DDX11L1,ENSG00000223972,Transcript,ENST00000456328,processed_transcript,,,,,,,
0,intron_variant&non_coding_transcript_variant,WASH7P,ENSG00000227232,Transcript,ENST00000488147,unprocessed_pseudogene,,10/10,,,,,
0,downstream_gene_variant,MIR6859-1,ENSG00000278267,Transcript,ENST00000619216,miRNA,,,,,,,
0,regulatory_region_variant,,,RegulatoryFeature,ENSR00000000002,open_chromatin_region,,,,,,,


In [22]:
#join splitted INFO column by bar (|) with main vcf file 
final_INFO_splitted_file = df1.join(p)

In [22]:
final_INFO_splitted_file.head()

,CHROM,POS,ID,REF,ALT,QUAL,FILTER,FORMAT,Sample_Count,INFO,...,Feature_type,Feature,BIOTYPE,EXON,INTRON,Existing_variation,SIFT,PolyPhen,AF,ExAC_AF
0,chr1,14522,.,G,A,118.64,PASS,GT:AD:DP:GQ:PL,2,ExcessHet=3.0103;FS=0;MQ=60;QD=33.16;SOR=2.303...,...,Transcript,ENST00000450305,transcribed_unprocessed_pseudogene,,,,,,,
0,chr1,14522,.,G,A,118.64,PASS,GT:AD:DP:GQ:PL,2,ExcessHet=3.0103;FS=0;MQ=60;QD=33.16;SOR=2.303...,...,Transcript,ENST00000456328,processed_transcript,,,,,,,
0,chr1,14522,.,G,A,118.64,PASS,GT:AD:DP:GQ:PL,2,ExcessHet=3.0103;FS=0;MQ=60;QD=33.16;SOR=2.303...,...,Transcript,ENST00000488147,unprocessed_pseudogene,,10/10,,,,,
0,chr1,14522,.,G,A,118.64,PASS,GT:AD:DP:GQ:PL,2,ExcessHet=3.0103;FS=0;MQ=60;QD=33.16;SOR=2.303...,...,Transcript,ENST00000619216,miRNA,,,,,,,
0,chr1,14522,.,G,A,118.64,PASS,GT:AD:DP:GQ:PL,2,ExcessHet=3.0103;FS=0;MQ=60;QD=33.16;SOR=2.303...,...,RegulatoryFeature,ENSR00000000002,open_chromatin_region,,,,,,,


In [23]:
len(final_INFO_splitted_file.axes[0])

79361523

In [24]:
final_INFO_splitted_file.to_csv('Asthma_splitted_mutations.csv')

In [25]:
# To remove any extra or unwanted columns from Arranged_cols file
removed_columns = final_INFO_splitted_file.drop(['INFO'],axis=1)

In [26]:
All_Asthma_mutation_unique = removed_columns.drop_duplicates()

In [28]:
len(All_Asthma_mutation_unique.axes[0])

6855669

In [27]:
#print output file with all mutations and its consequences
All_Asthma_mutation_unique.to_csv('Asthma_final_filtered_mutations.csv')